Script with the functions of finals GLMs used to create L-HAZELNUT model. 

In [ ]:
import pandas as pd
import statsmodels.api as sm
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import norm


In [ ]:
def length_nbnodes(shoot):
    # Creazione delle colonne polinomiali
    shoot['length_05'] = shoot['length']**0.5
    shoot['length_2'] = shoot['length']**2

    # Definizione delle variabili indipendenti e dipendenti
    X = shoot[['length_05', 'length_2']]
    y = shoot['node']

    # Aggiunta dell'intercetta (costante) al modello
    X = sm.add_constant(X, has_constant='add')

    # Creazione e addestramento del modello di regressione lineare
    model = sm.OLS(y, X).fit()

    # Stampa del sommario del modello
    sum = model.summary()
    print(sum)

    # Calcolo dell'AIC del modello
    aic = model.aic
    print("AIC:", aic)

    return model

In [ ]:
def plot_length_nbnodes(shoot, model, xlim=None):
    # Creazione delle colonne polinomiali
    shoot['length_05'] = shoot['length']**0.5
    shoot['length_2'] = shoot['length']**2

    # Creazione del nuovo asse x per le previsioni
    if xlim is None:
        xlim = (0, shoot['length'].max())
    new_x = np.linspace(xlim[0], xlim[1], 100)
    
    # Creazione del DataFrame per le nuove previsioni
    new_data = pd.DataFrame({
        'length': new_x,
        'length_05': new_x**0.5,
        'length_2': new_x**2
    })
    
    new_data = sm.add_constant(new_data, has_constant='add')

    # Previsioni dal modello
    pred = model.get_prediction(new_data)
    summary_frame = pred.summary_frame(alpha=0.05)
    new_y = summary_frame['mean']
    upr = summary_frame['mean_ci_upper']
    lwr = summary_frame['mean_ci_lower']
    
    # Creazione del grafico
    plt.figure(figsize=(12, 9))
    plt.scatter(shoot['length'], shoot['node'], color='blue', label='Observed data')
    plt.plot(new_x, new_y, color='black', linewidth=2, label='Fitted line')
    plt.fill_between(new_x, lwr, upr, color='grey', alpha=0.5, label='95% Prediction interval')
    
    plt.xlabel('length(cm)')
    plt.ylabel('length(node)')
    plt.title('length(cm) vs length(node)')
    plt.legend()
    plt.show()